In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
train_dir='drive/My Drive/TinyML_Cookbook/Nano_desk_objects/dataset/train'

import tensorflow as tf

ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    subset='both',
    validation_split=0.2,
    seed=123,
    interpolation='bilinear',
    image_size=(48,48)
)
train_ds = ds[0]
val_ds = ds[1]

Found 74 files belonging to 3 classes.
Using 60 files for training.
Using 14 files for validation.


In [3]:
class_names = train_ds.class_names
num_classes = len(class_names)

rescale = tf.keras.layers.Rescaling(1./255, offset=-1)
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds = val_ds.map(lambda x, y: (rescale(x), y))

In [4]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

base_model = MobileNetV2(input_shape=(48,48,3), include_top=False, weights='imagenet',
                         alpha=0.35)

base_model.trainable = False
feat_extr = base_model

<ipython-input-4-0f7a9a3e3172>:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(48,48,3), include_top=False, weights='imagenet',


2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
print('num. weights:', len(base_model.weights))
print('num. trainable weights:', len(base_model.trainable_weights))
print('num. non-trainable weights:', len(base_model.non_trainable_weights))

num. weights: 260
num. trainable weights: 0
num. non-trainable weights: 260


In [6]:
layers = tf.keras.layers
augmen = tf.keras.Sequential([
    layers.RandomRotation(0.2),
    layers.RandomFlip('horizontal')
])

train_ds = train_ds.map(lambda x, y: (augmen(x), y))
val_ds = val_ds.map(lambda x, y: (augmen(x), y))

In [7]:
global_avg_layer = layers.GlobalAveragePooling2D()
dense_layer = layers.Dense(num_classes, activation='softmax')

x = global_avg_layer(feat_extr.layers[-1].output)
x = layers.Dropout(0.2)(x)
outputs = dense_layer(x)

model = tf.keras.Model(inputs=feat_extr.inputs,
                       outputs=outputs)

In [8]:
lr = 0.0005
opt_f = tf.keras.optimizers.Adam(learning_rate=lr)
loss_f = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer=opt_f,
              loss=loss_f,
              metrics=['accuracy'])

In [9]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=20)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 48, 48, 3))
  warnings.warn(msg)


2/2 ━━━━━━━━━━━━━━━━━━━━ 18s 7s/step - accuracy: 0.5639 - loss: 0.7871 - val_accuracy: 0.5714 - val_loss: 0.7078
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 307ms/step - accuracy: 0.6410 - loss: 0.6670 - val_accuracy: 0.7143 - val_loss: 0.9645
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 299ms/step - accuracy: 0.6646 - loss: 0.6121 - val_accuracy: 0.5000 - val_loss: 0.8242
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 318ms/step - accuracy: 0.7069 - loss: 0.7074 - val_accuracy: 0.5714 - val_loss: 0.7733
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 295ms/step - accuracy: 0.6632 - loss: 0.6859 - val_accuracy: 0.6429 - val_loss: 0.6518
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 332ms/step - accuracy: 0.7493 - loss: 0.4997 - val_accuracy: 0.7857 - val_loss: 0.6084
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 331ms/step - accuracy: 0.6306 - loss: 0.7791 - val_accuracy: 0.6429 - val_loss: 0.6077
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step - accuracy: 0.7826 - loss: 0.4857 - val_accuracy: 0.7857 - val_loss: 0.5282
Epoch

In [10]:
model.export('desk_objects_recognition')

Saved artifact at 'desk_objects_recognition'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 48, 48, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  132979145377936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132979145380816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132979145380432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132979145380240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132979145377744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132979145381392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132979145382160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132979145378320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132979145378128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132979145381776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13297914

In [11]:
test_dir = 'drive/My Drive/TinyML_Cookbook/Nano_desk_objects/dataset/test'
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    interpolation='bilinear',
    image_size=(48,48)
)

test_ds = test_ds.map(lambda x, y: (rescale(x), y))

repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(60):
    yield [i_value]

TF_MODEL = 'desk_objects_recognition'

converter = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL)
converter.representative_dataset = tf.lite.RepresentativeDataset(representative_data_gen)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8

tfl_model = converter.convert()

Found 15 files belonging to 2 classes.


In [12]:
interp = tf.lite.Interpreter(model_content=tfl_model)

interp.allocate_tensors()

i_details = interp.get_input_details()[0]
o_details = interp.get_output_details()[0]

i_quant = i_details['quantization_parameters']
i_scale = i_quant['scales'][0]
i_zero_point = i_quant['zero_points'][0]

In [13]:
import numpy as np

test_ds0 = test_ds.unbatch()

num_correct_samples = 0
num_samples = len(list(test_ds0.batch(1)))

for i_value, o_value in test_ds0.batch(1):
  i_value = (i_value/i_scale) + i_zero_point
  i_value = tf.cast(i_value, dtype=tf.int8)
  interp.set_tensor(i_details['index'], i_value)
  interp.invoke()

  o_pred = interp.get_tensor(o_details['index'])[0]
  if np.argmax(o_pred) == o_value:
    num_correct_samples += 1

print('Accuracy:', num_correct_samples/num_samples)

Accuracy: 0.6


In [14]:
open('model.tflite', 'wb').write(tfl_model)
!apt-get update && apt-get -qq install xxd
!pip install netron
!xxd -i model.tflite > model.h
!sed -i 's/unsigned char/const unsigned char/g' model.h
!sed -i 's/const/alignas(8) const/g' model.h

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,844 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,244 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,118 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,605 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]


In [19]:
size_tfl_model = len(tfl_model)
print(size_tfl_model, "bytes")

@tf.function
def func(x):
    return model(x)

model_func = func.get_concrete_function(np.zeros((0,48,48,3), dtype=np.uint8))
ops = model_func.graph.get_operations()

unique_ops = set()

for op in ops:
    unique_ops.add(op)

for op in unique_ops:
    print(f'Name: {op.name}')
    print(f'Op: {op.type}')
    print()

619840 bytes


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(0, 48, 48, 3))
  warnings.warn(msg)


Name: functional_1_1/expanded_conv_depthwise_BN_1/batchnorm/Rsqrt
Op: Rsqrt

Name: functional_1_1/block_9_project_BN_1/Cast_1/ReadVariableOp/resource
Op: Placeholder

Name: functional_1_1/global_average_pooling2d_1/Mean
Op: Mean

Name: functional_1_1/block_9_project_1/convolution/ReadVariableOp/resource
Op: Placeholder

Name: functional_1_1/dense_1/BiasAdd
Op: BiasAdd

Name: functional_1_1/expanded_conv_depthwise_BN_1/Cast_2/ReadVariableOp/resource
Op: Placeholder

Name: functional_1_1/block_2_depthwise_BN_1/batchnorm/add_1
Op: AddV2

Name: functional_1_1/block_9_project_BN_1/batchnorm/add
Op: AddV2

Name: functional_1_1/dense_1/BiasAdd/ReadVariableOp
Op: ReadVariableOp

Name: functional_1_1/block_9_project_BN_1/Cast/ReadVariableOp/resource
Op: Placeholder

Name: functional_1_1/out_relu_1/Relu6
Op: Relu6

Name: functional_1_1/expanded_conv_depthwise_BN_1/Cast_1/ReadVariableOp/resource
Op: Placeholder

Name: functional_1_1/block_9_project_BN_1/Cast_2/ReadVariableOp/resource
Op: Placehol